<a href="https://colab.research.google.com/github/edwardleetenafly/LA-net/blob/main/lanet_week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
!pip install nibabel scipy scikit-image tqdm


In [13]:
import nibabel as nib
import numpy as np
from pathlib import Path

root = Path("/content/drive/MyDrive/la_net")

# Pick one case
case_id = "la_003"
img = nib.load(root / "imagesTr" / f"{case_id}.nii")
msk = nib.load(root / "labelsTr" / f"{case_id}.nii")

img_data = img.get_fdata()
msk_data = msk.get_fdata()

print("Image shape:", img_data.shape)
print("Mask shape:", msk_data.shape)
print("Image intensity range:", img_data.min(), img_data.max())

print("Voxel spacing:", img.header.get_zooms())


Image shape: (320, 320, 130)
Mask shape: (320, 320, 130)
Image intensity range: 0.0 1999.0
Voxel spacing: (np.float32(1.25), np.float32(1.25), np.float32(1.37))


In [14]:
import scipy.ndimage as ndimage

def resample_to_isotropic(volume, spacing, new_spacing=(1,1,1), order=1):
    zoom = np.array(spacing) / np.array(new_spacing)
    return ndimage.zoom(volume, zoom, order=order)

img_spacing = img.header.get_zooms()
msk_spacing = msk.header.get_zooms()

img_resampled = resample_to_isotropic(img_data, img_spacing, new_spacing=(1,1,1), order=1)
msk_resampled = resample_to_isotropic(msk_data, msk_spacing, new_spacing=(1,1,1), order=0)

print("Resampled image shape:", img_resampled.shape)
print("Resampled mask shape:", msk_resampled.shape)


Resampled image shape: (400, 400, 178)
Resampled mask shape: (400, 400, 178)


In [15]:
import numpy as np

def normalize(volume):
    mean = np.mean(volume)
    std = np.std(volume)
    return (volume - mean) / (std + 1e-8)

# img_resampled must already exist from Wednesday
img_norm = normalize(img_resampled)

print("Mean after normalization:", img_norm.mean())
print("Std after normalization:", img_norm.std())


Mean after normalization: 1.0037653604679312e-15
Std after normalization: 0.9999999999617678


In [16]:
import numpy as np

def pad_or_crop(volume, target_shape=(128,128,128)):
    out = np.zeros(target_shape, dtype=volume.dtype)

    orig = np.array(volume.shape)
    targ = np.array(target_shape)

    # start index in original
    start = np.maximum((orig - targ)//2, 0)
    # start index in output
    paste_start = np.maximum((targ - orig)//2, 0)

    # slices for original and output
    src = tuple(slice(s, s + min(o, t)) for s, o, t in zip(start, orig, targ))
    dst = tuple(slice(p, p + (ss.stop - ss.start)) for p, ss in zip(paste_start, src))

    out[dst] = volume[src]
    return out

# img_norm (from Thursday) and msk_resampled (from Wednesday) must exist
img_final = pad_or_crop(img_norm)
msk_final = pad_or_crop(msk_resampled)

print("Final image shape:", img_final.shape)
print("Final mask shape:", msk_final.shape)


Final image shape: (128, 128, 128)
Final mask shape: (128, 128, 128)


In [17]:
from pathlib import Path
import numpy as np

prep_dir = Path("/content/drive/MyDrive/la_net/preprocessed")
prep_dir.mkdir(exist_ok=True)

case_id = "la_003"   # change if testing other cases

np.save(prep_dir / f"{case_id}_image.npy", img_final.astype(np.float32))
np.save(prep_dir / f"{case_id}_mask.npy", msk_final.astype(np.uint8))

print("Saved:", f"{case_id}_image.npy and {case_id}_mask.npy")


Saved: la_003_image.npy and la_003_mask.npy


In [18]:
import nibabel as nib
import numpy as np
from pathlib import Path
import scipy.ndimage as ndimage
import pandas as pd
from tqdm import tqdm

root = Path("/content/drive/MyDrive/la_net")
manifest = pd.read_csv(root / "manifest.csv")

prep_dir = root / "preprocessed"
prep_dir.mkdir(exist_ok=True)

# ----------------------------------------------------------
# 1. Resample to isotropic spacing
# ----------------------------------------------------------
def resample_to_isotropic(volume, spacing, new=(1,1,1), order=1):
    zoom = np.array(spacing) / np.array(new)
    return ndimage.zoom(volume, zoom, order=order)


# ----------------------------------------------------------
# 2. Pad or crop to final fixed shape
# ----------------------------------------------------------
def pad_or_crop(volume, target=(128,128,128)):
    out = np.zeros(target, dtype=volume.dtype)

    orig = np.array(volume.shape)
    targ = np.array(target)

    # start index in original volume
    start = np.maximum((orig - targ)//2, 0)
    # start index in output
    paste_start = np.maximum((targ - orig)//2, 0)

    # slices
    src = tuple(slice(s, s + min(o, t)) for s, o, t in zip(start, orig, targ))
    dst = tuple(slice(p, p + (ss.stop - ss.start)) for p, ss in zip(paste_start, src))

    out[dst] = volume[src]
    return out


# ----------------------------------------------------------
# 3. Normalize AFTER cropping (correct order!)
# ----------------------------------------------------------
def normalize(volume):
    mean = np.mean(volume)
    std = np.std(volume)
    return (volume - mean) / (std + 1e-8)


# ----------------------------------------------------------
# 4. Preprocessing loop over all cases
# ----------------------------------------------------------
for _, row in tqdm(manifest.iterrows(), total=len(manifest)):
    cid = row["case_id"]

    # load original NIfTI
    img = nib.load(row["image_path"])
    msk = nib.load(row["mask_path"])

    img_d = img.get_fdata()
    msk_d = msk.get_fdata()

    # --- RESAMPLE ---
    img_r = resample_to_isotropic(img_d, img.header.get_zooms(), order=1)
    msk_r = resample_to_isotropic(msk_d, msk.header.get_zooms(), order=0)

    # --- PAD/CROP BEFORE NORMALIZATION ---
    img_c = pad_or_crop(img_r)
    msk_c = pad_or_crop(msk_r)

    # --- NORMALIZE FINAL VOLUME ---
    img_f = normalize(img_c)

    # masks must remain 0 or 1
    msk_f = msk_c.astype(np.uint8)

    # --- SAVE ---
    np.save(prep_dir / f"{cid}_image.npy", img_f.astype(np.float32))
    np.save(prep_dir / f"{cid}_mask.npy",  msk_f)

print("Finished Week 2 preprocessing with correct normalization order.")


100%|██████████| 20/20 [00:57<00:00,  2.86s/it]

Finished Week 2 preprocessing with correct normalization order.


In [21]:
import numpy as np

arr = np.load("/content/drive/MyDrive/la_net/preprocessed/la_003_image.npy")
print("Shape:", arr.shape)
print("Dtype:", arr.dtype)
print("Min/Max:", arr.min(), arr.max())
print("Mean:", arr.mean(), "Std:", arr.std())



Shape: (128, 128, 128)
Dtype: float32
Min/Max: -1.6925095 3.4350688
Mean: 3.4924597e-08 Std: 0.99999994
